# Analyzing Startup Fundraising Deals from Crunchbase

In this guided project by Dataquest (DQ), I analyzed information about startup investments from [Crunchbase](https://www.crunchbase.com/). Crunchbase is "a website that crowdsources information on the fundraising rounds of many startups". I downloaded the data set (in `csv` format) from [here](https://github.com/datahoarder/crunchbase-october-2013/blob/master/crunchbase-investments.csv). The data set was extracted from Crunchbase in October 2013. 

My goal in this project is to provide answers to the following questions:
* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? How do these values compare with the proportions of the bottom 10% and bottom 1% raised?
* Which category of startup companies attracted the most investments?
* Which investor contributed the most money across all startups?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

I worked on the data set under the following conditions/constraints:
* 10 MB of available memory - this required me to lower the memory footprint of the data set and process it in chunks
* use a database like SQLite to represent the data set on disk, then utilize pandas to explore and analyze a subset of the data set in memory (SQL/pandas workflow) - this is a suggested method of handling large data sets in Python

## Data Set Exploration

I first explored the first 5 rows of the data set to look into the data and its columns. 

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 99

first_5_rows = pd.read_csv('crunchbase-investments.csv', nrows=5)
print('Total number of columns =', len(first_5_rows.columns))
first_5_rows.head()

Total number of columns = 20


,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012,2000000
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012,20000
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012,20000
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012,20000
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011,20000


The data set has 20 columns of data and is a mix of numeric and string types. String columns `company_permalink` and `investor_permalink` represent URLs and can be removed since they are not useful for data analysis. 

Next, I determined the number of missing values and the memory footprint of each data column. I also determined the number of rows of the data set. Since there is a constraint of 10 MB of available memory, I processed the data set in chunks. The data set requires the `ISO-8859-1` encoding for reading. 

In [2]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1')
col_null = []
col_mem = []
rows = 0

for chunk in chunk_iter:
    col_null.append(chunk.isnull().sum())
    col_mem.append(chunk.memory_usage(deep=True, index=False))
    rows += len(chunk)

col_null_cc = pd.concat(col_null)
col_mem_cc = pd.concat(col_mem)

col_null_fin = col_null_cc.groupby(col_null_cc.index).sum().sort_values(ascending=False)
col_mem_fin = col_mem_cc.groupby(col_mem_cc.index, sort=True).sum()/2**20

print('Total number of rows =', rows)
col_null_fin

Total number of rows = 52870


investor_category_code    50427
investor_state_code       16809
investor_city             12480
investor_country_code     12001
raised_amount_usd          3599
company_category_code       643
company_city                533
company_state_code          492
funding_round_type            3
funded_year                   3
funded_month                  3
funded_at                     3
funded_quarter                3
investor_name                 2
investor_permalink            2
investor_region               2
company_region                1
company_permalink             1
company_name                  1
company_country_code          1
dtype: int64

The data set has 52870 rows. Columns `investor_category_code`, `investor_state_code`, `investor_city`, and `investor_country_code` have the most number of missing values. 

In [3]:
print(col_mem_fin.round(decimals=1).sort_values(ascending=False))
print('\n')
print('The total memory footprint of the dataset is {} MB'.format(round(col_mem_fin.sum(),1)))

investor_permalink        4.7
company_permalink         3.9
investor_name             3.7
company_name              3.4
funded_at                 3.4
company_category_code     3.3
company_city              3.3
company_region            3.3
funding_round_type        3.3
funded_month              3.2
funded_quarter            3.2
investor_region           3.2
company_state_code        3.0
company_country_code      3.0
investor_city             2.8
investor_country_code     2.5
investor_state_code       2.4
investor_category_code    0.6
funded_year               0.4
raised_amount_usd         0.4
dtype: float64


The total memory footprint of the dataset is 57.0 MB


String columns `investor_permalink`, `company_permalink`, and `investor_name` consume the largest amount of memory. The total memory footprint of the data set (index not included) is 57 MB.  

To lower the memory consumption of the data set, I first dropped the columns not useful for data analysis (`company_permalink` and `investor_permalink`). I also removed the columns with missing values of greater than 90%. 

In [4]:
drop_cols = ['company_permalink','investor_permalink']
for col in first_5_rows.columns:
    if col_null_fin[col] >= 0.9*rows:
        drop_cols.append(col)

keep_cols = first_5_rows.columns.drop(drop_cols)
keep_cols

Index(['company_name', 'company_category_code', 'company_country_code',
       'company_state_code', 'company_region', 'company_city', 'investor_name',
       'investor_country_code', 'investor_state_code', 'investor_region',
       'investor_city', 'funding_round_type', 'funded_at', 'funded_month',
       'funded_quarter', 'funded_year', 'raised_amount_usd'],
      dtype='object')

## Memory Optimization

To further decrease the memory footprint of the data set, I needed to optimize the data types. I first identified the data type of each column. 

In [5]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)
col_dtypes = {}

for chunk in chunk_iter:
    for col in chunk.columns:
        if col not in col_dtypes:
            col_dtypes[col] = [str(chunk.dtypes[col])]
        else:
            col_dtypes[col].append(str(chunk.dtypes[col]))

col_dtypes_set = {}
for k in col_dtypes.keys():
    col_dtypes_set[k] = set(col_dtypes[k])

col_dtypes_set

{'company_name': {'object'},
 'company_category_code': {'object'},
 'company_country_code': {'object'},
 'company_state_code': {'object'},
 'company_region': {'object'},
 'company_city': {'object'},
 'investor_name': {'object'},
 'investor_country_code': {'float64', 'object'},
 'investor_state_code': {'float64', 'object'},
 'investor_region': {'object'},
 'investor_city': {'float64', 'object'},
 'funding_round_type': {'object'},
 'funded_at': {'object'},
 'funded_month': {'object'},
 'funded_quarter': {'object'},
 'funded_year': {'float64', 'int64'},
 'raised_amount_usd': {'float64'}}

String columns are identified as `object` type, while the numeric columns as `float64` or `int64` types. The inclusion of `float64` type in the string columns `investor_country_code`, `investor_state_code`, and `investor_city` is due to the missing values. `float64` type is also included in the numeric column `funded_year` due to the missing values.  

I now inspected the numeric column `raised_amount_usd` to see if I can represent it as an integer of low-memory type. 

In [6]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

min_val = 0
max_val = 0
for chunk in chunk_iter:
    if chunk['raised_amount_usd'].max() > max_val:
        max_val = chunk['raised_amount_usd'].max()
    
    if chunk['raised_amount_usd'].min() < min_val:
        min_val = chunk['raised_amount_usd'].min()
          
print(max_val)
print(min_val)

3200000000.0
0


The range of values in the numeric column `raised_amount_usd` is from 0 to 3200000000. Also, the first 5 rows of the data set shown earlier indicate the values in this column to be whole numbers. Therefore, I can represent this data column as integers. 

For the string columns, I can cast them as of `category` type if they have low cardinality, e.g. [the number of unique values is lower than 50% of the total count](https://medium.com/@vincentteyssier/optimizing-the-size-of-a-pandas-dataframe-for-low-memory-environment-5f07db3d72e). I first obtained the number of unique values per string column.  

In [7]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

col_unique_vals = {} 
for chunk in chunk_iter:
    str_chunk = chunk.select_dtypes(include='object')
    str_chunk_cols = str_chunk.columns

    for col in str_chunk_cols:
        unique = str_chunk[col].value_counts()
        
        if col in col_unique_vals:
            col_unique_vals[col].update(unique.index)
        else:
            col_unique_vals[col] = set(unique.index) 

for col in col_unique_vals:
    print(col, ":", len(col_unique_vals[col]))


company_name : 11573
company_category_code : 43
company_country_code : 2
company_state_code : 50
company_region : 546
company_city : 1229
investor_name : 10465
investor_country_code : 72
investor_state_code : 50
investor_region : 585
investor_city : 990
funding_round_type : 9
funded_at : 2808
funded_month : 192
funded_quarter : 72


From the results, I cast the following string columns as categorical: `company_category_code`, `company_country_code`, `company_state_code`, `company_region`, `company_city`, `investor_country_code`, `investor_state_code`, `investor_region`, `investor_city`, and `funding_round_type`. 

In [8]:
category_cols = {'company_category_code' : 'category', 'company_country_code' : 'category',
                'company_state_code' : 'category', 'company_region': 'category', 'company_city' : 'category',
                'investor_country_code' : 'category', 'investor_state_code' : 'category', 'investor_region' : 'category',
                'investor_city' : 'category', 'funding_round_type' : 'category'}

As for the remaining data columns:

* Due to the format of values in the string column `funded_at` (as initially shown in the first 5 rows of the data set), I intended to parse it into `datetime` type.
* String columns `funded_month` and `funded_quarter` can be cleaned, e.g. remove the year value, and then cast as categorical as long as their year value is similar to the values in the numeric column `funded_year`. I checked the similarity using the code below. 

In [9]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols)

for chunk in chunk_iter:
    chunk = chunk.dropna(subset=['funded_month', 'funded_quarter', 'funded_year'])
    
    year_month = chunk['funded_month'].str.split("-", n=1, expand=True)[0].astype('int')
    year_quarter = chunk['funded_quarter'].str.split("-", n=1, expand=True)[0].astype('int')
    year_year = chunk['funded_year'].astype('int')
    
    if not (year_year.equals(year_month) or year_year.equals(year_quarter)):
        print('Year in column `funded_month` and/or `funded_quarter` not equal with `funded_year`!')

Since there is no printed output to indicate differences in the year values of `funded_month` and `funded_quarter` versus `funded_year`, I proceeded to the cleaning of the mentioned data columns. 

Lastly, the numeric column `funded_year`  can be cast as an integer. 

To summarize, I performed the following to reduce the overall memory of the data set from 57 MB to under 10 MB:

* Dropped the string columns `company_permalink` and `investor_permalink` from the data set
* Removed the data columns with missing values of greater than 90% (`investor_category_code`)
* Filled the missing values in the numeric column `raised_amount_usd` with zero then cast the column as an integer
* Cast the string columns of low cardinality (`company_category_code`, `company_country_code`, `company_state_code`, `company_region`, `company_city`, `investor_country_code`, `investor_state_code`, `investor_region`, `investor_city`, and `funding_round_type`) as type `category`
* Parsed the string column `funded_at` into `datetime` type
* Cleaned the string columns `funded_month` and `funded_quarter` and cast them as type `category`
* Cast the numeric column `funded_year` as integer 

In [10]:
col_mem2 = [] # Memory of the data set after modification
rows2 = 0 # Number of rows in the data set after modification

chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols, 
                         dtype=category_cols, parse_dates=['funded_at'])

for chunk in chunk_iter:
    
    # Drop the rows with null values in 'funded_month', 'funded_quarter', and 'funded_year' columns
    chunk = chunk.dropna(subset=['funded_month', 'funded_quarter', 'funded_year'])
    
    # Fill the missing values in 'raised_amount_usd' column with zero, then convert to integer type  
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0).astype('int')
    
    # Clean the 'funded_month' column, retain the month value, and convert to category type
    chunk['funded_month'] = chunk['funded_month'].str.split("-", n=1, expand=True)[1].astype('category')
    
    # Clean the 'funded_quarter' column, retain the quarter value, and convert to category type
    chunk['funded_quarter'] = chunk['funded_quarter'].str.split("-", n=1, expand=True)[1].astype('category')
    
    # 'funded_year' column to optimum integer type
    chunk['funded_year'] = pd.to_numeric(chunk['funded_year'], downcast='integer')  
    
    col_mem2.append(chunk.memory_usage(deep=True, index=False))
    rows2 += len(chunk)

col_mem2_cc = pd.concat(col_mem2)

col_mem2_fin = col_mem2_cc.groupby(col_mem2_cc.index, sort=True).sum()/2**20

print(col_mem2_fin.round(decimals=1).sort_values(ascending=False))
print('\n')
print('The total memory footprint of the dataset is {} MB'.format(round(col_mem2_fin.sum(),1)))
print('Total number of rows is {}'.format(rows2))

investor_name            3.7
company_name             3.4
company_city             0.6
funded_at                0.4
investor_city            0.3
company_region           0.3
raised_amount_usd        0.2
investor_region          0.2
company_country_code     0.1
company_state_code       0.1
funded_quarter           0.1
funded_month             0.1
investor_state_code      0.1
funded_year              0.1
funding_round_type       0.1
investor_country_code    0.1
company_category_code    0.1
dtype: float64


The total memory footprint of the dataset is 9.9 MB
Total number of rows is 52867


The final data types of the columns are shown below. 

In [11]:
chunk.dtypes

company_name                     object
company_category_code          category
company_country_code           category
company_state_code             category
company_region                 category
company_city                   category
investor_name                    object
investor_country_code          category
investor_state_code            category
investor_region                category
investor_city                  category
funding_round_type             category
funded_at                datetime64[ns]
funded_month                   category
funded_quarter                 category
funded_year                       int16
raised_amount_usd                 int32
dtype: object

This is the form of the data set before loading into a table in my SQLite database.  

In [12]:
chunk.sample(n=5)

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
52470,CrowdFlower,enterprise,USA,CA,SF Bay,San Francisco,Travis Kalanick,NaN,NaN,unknown,NaN,angel,2009-03-01,03,Q1,2009,1200000
52295,Bislr,software,USA,CA,SF Bay,San Francisco,Timothy Draper,NaN,NaN,unknown,NaN,series-a,2013-02-13,02,Q1,2013,3500000
52153,EyeVerify,security,USA,KS,Kansas City,Kansas City,Thad Langford,NaN,NaN,unknown,NaN,series-a,2012-05-25,05,Q2,2012,1500000
51053,BizeeBee,software,USA,CA,SF Bay,Palo Alto,Rob Classeen,NaN,NaN,unknown,NaN,angel,2011-04-28,04,Q2,2011,435000
50767,ZenPayroll,software,USA,CA,SF Bay,San Francisco,Ram Shriram,NaN,NaN,unknown,NaN,series-a,2012-12-12,12,Q4,2012,6100000


## Data Migration to SQLite

Before migrating the data set to SQLite, I created and connected to a new database. 

In [13]:
import sqlite3
conn = sqlite3.connect('crunchbase.db')
cur = conn.cursor()

I then created a new table `crunch` with data columns corresponding to the data set. 

In [14]:
q1 = '''
    CREATE TABLE crunch (
        company_name TEXT,
        company_category_code TEXT,
        company_country_code TEXT,
        company_state_code TEXT,
        company_region TEXT,
        company_city TEXT,
        investor_name TEXT,
        investor_country_code TEXT,
        investor_state_code TEXT,
        investor_region TEXT,
        investor_city TEXT, 
        funding_round_type TEXT,
        funded_at DATE,
        funded_month SMALLINT,
        funded_quarter TEXT,
        funded_year SMALLINT,
        raised_amount_usd BIGINT
    );
'''

cur.execute(q1)
conn.commit()

I exported each chunk of my modified data set into the new SQLite table. 

In [15]:
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=keep_cols, 
                         dtype=category_cols, parse_dates=['funded_at'])

for chunk in chunk_iter:
    
    chunk = chunk.dropna(subset=['funded_month', 'funded_quarter', 'funded_year'])
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0).astype('int')
    chunk['funded_month'] = chunk['funded_month'].str.split("-", n=1, expand=True)[1].astype('category')
    chunk['funded_quarter'] = chunk['funded_quarter'].str.split("-", n=1, expand=True)[1].astype('category')
    chunk['funded_year'] = pd.to_numeric(chunk['funded_year'], downcast='integer') 
    
    chunk.to_sql('crunch', conn, if_exists='append', index=False)

I then queried the table to check on the data types of each column. 

In [16]:
table_info = pd.read_sql('PRAGMA table_info(crunch)', conn)
table_info

,cid,name,type,notnull,dflt_value,pk
0,0,company_name,TEXT,0,None,0
1,1,company_category_code,TEXT,0,None,0
2,2,company_country_code,TEXT,0,None,0
3,3,company_state_code,TEXT,0,None,0
4,4,company_region,TEXT,0,None,0
5,5,company_city,TEXT,0,None,0
6,6,investor_name,TEXT,0,None,0
7,7,investor_country_code,TEXT,0,None,0
8,8,investor_state_code,TEXT,0,None,0
9,9,investor_region,TEXT,0,None,0


The file size of the database is around 7 MB as shown below. I also presented a small subset of the data from the database table. 

In [17]:
import os
print('File size of the database =',os.path.getsize('crunchbase.db'), 'bytes')

File size of the database = 7286784 bytes


In [18]:
sample_data = pd.read_sql('SELECT * FROM crunch LIMIT 5;', conn)
sample_data

,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_name,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,AdverCar,advertising,USA,CA,SF Bay,San Francisco,1-800-FLOWERS.COM,USA,NY,New York,New York,series-a,2012-10-30 00:00:00,10,Q4,2012,2000000
1,LaunchGram,news,USA,CA,SF Bay,Mountain View,10Xelerator,USA,OH,Columbus,Columbus,other,2012-01-23 00:00:00,1,Q1,2012,20000
2,uTaP,messaging,USA,None,United States - Other,None,10Xelerator,USA,OH,Columbus,Columbus,other,2012-01-01 00:00:00,1,Q1,2012,20000
3,ZoopShop,software,USA,OH,Columbus,columbus,10Xelerator,USA,OH,Columbus,Columbus,angel,2012-02-15 00:00:00,2,Q1,2012,20000
4,eFuneral,web,USA,OH,Cleveland,Cleveland,10Xelerator,USA,OH,Columbus,Columbus,other,2011-09-08 00:00:00,9,Q3,2011,20000


## Data Exploration and Analysis Using SQL/Pandas Workflow

I used the SQL/Pandas workflow to answer questions related to startup investments:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? How do these values compare with the proportions of the bottom 10% and bottom 1% raised?
* Which category of startup companies attracted the most investments?
* Which investor contributed the most money across all startups?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

The SQL/pandas workflow involves the use of SQL for data selection and pandas for iterative exploration and analysis. 

*What proportion of the total amount of funds did the top 10% raise? What about the top 1%? How do these values compare with the proportions of the bottom 10% and bottom 1% raised?*

* As shown below, the top 10% of startup companies raised 63% of the total amount of funds; the top 1% raised 22% of the total funds. The contributions of the bottom 10% and 1% of the startup companies are negligible (less than 1% of the total amount of funds). 

In [19]:
q2 = '''
    SELECT 
        company_name,
        raised_amount_usd
    FROM crunch
    WHERE raised_amount_usd != 0;
'''

df2 = pd.read_sql(q2, conn)
df2_fin = df2.groupby('company_name', sort=False).sum().sort_values(ascending=False, by=['raised_amount_usd'])
top10 = int(len(df2_fin)*0.1)
top1 = int(len(df2_fin)*0.01)

top10_funds = df2_fin.iloc[0:top10-1]['raised_amount_usd'].sum()
top1_funds = df2_fin.iloc[0:top1-1]['raised_amount_usd'].sum()
bot10_funds = df2_fin.iloc[-top10:-1]['raised_amount_usd'].sum()
bot1_funds = df2_fin.iloc[-top1:-1]['raised_amount_usd'].sum()
tot_funds = df2_fin['raised_amount_usd'].sum()

d = {
    'Companies' : ['All', 'Top 10%', 'Top 1%', 'Bottom 10%', 'Bottom 1%'],
    'Amount of funds, USD' : [tot_funds, top10_funds, top1_funds, bot10_funds, bot1_funds],
    'Proportion of total funds' : [tot_funds, top10_funds, top1_funds, bot10_funds, bot1_funds]/tot_funds
}

df2_funds = pd.DataFrame(data=d)
df2_funds

,Companies,"Amount of funds, USD",Proportion of total funds
0,All,650247286085,1.000000
1,Top 10%,410649278526,0.631528
2,Top 1%,141368441740,0.217407
3,Bottom 10%,181454331,0.000279
4,Bottom 1%,1461100,0.000002


*Which category of startup companies attracted the most investments?*

* `Biotech` companies attracted the most investments, followed by `software` and `cleantech`. 

In [20]:
q3 = '''
    SELECT 
        company_category_code,
        raised_amount_usd
    FROM crunch
    WHERE raised_amount_usd != 0;
'''

df3 = pd.read_sql(q3, conn)
df3_fin = df3.groupby('company_category_code', sort=False).sum().sort_values(ascending=False, by=['raised_amount_usd'])
df3_fin.head(10)

,raised_amount_usd
company_category_code,
biotech,110396423062
software,73084516724
cleantech,52705225028
enterprise,45860927273
web,40143264989
mobile,38039961512
medical,25367105281
advertising,25076661879
ecommerce,22567220071


*Which investor contributed the most money across all startups?*

* `Kleiner Perkins Caufield & Byers` contributed the most money across all startups. 

In [21]:
q4 = '''
    SELECT 
        investor_name,
        raised_amount_usd
    FROM crunch
    WHERE raised_amount_usd != 0;
'''

df4 = pd.read_sql(q4, conn)
df4_fin = df4.groupby('investor_name', sort=False).sum().sort_values(ascending=False, by=['raised_amount_usd'])
df4_fin.head(10)

,raised_amount_usd
investor_name,
Kleiner Perkins Caufield & Byers,11217826376
New Enterprise Associates,9692542344
Accel Partners,6472126199
Goldman Sachs,6375459000
Sequoia Capital,6039402410
Greylock Partners,4960982939
Intel Capital,4695616755
Draper Fisher Jurvetson (DFJ),4501461238
Oak Investment Partners,4350064727


*Which investors contributed the most money per startup?*

* `Sprint Nextel`, `GI Partners`, and `Eagle River Holdings` have contributed the most money per startup. 

In [22]:
q5 = '''
    SELECT 
        investor_name,
        SUM(raised_amount_usd) USD,
        COUNT(DISTINCT company_name) startups
    FROM crunch
    WHERE raised_amount_usd != 0
    GROUP BY investor_name
    ORDER BY 2 DESC;
'''

df5 = pd.read_sql(q5, conn)
df5['USD per startup'] = df5['USD']/df5['startups']
df5.sort_values(ascending=False, by=['USD per startup']).head(10)

,investor_name,USD,startups,USD per startup
32,Sprint Nextel,2500000000,1,2.500000e+09
109,GI Partners,1050000000,1,1.050000e+09
34,Eagle River Holdings,2457000000,3,8.190000e+08
167,Siemens PLM Software,750000000,1,7.500000e+08
23,Digital Sky Technologies,2921807000,6,4.869678e+08
290,Laurel Crown Partners,450000000,1,4.500000e+08
343,Beck Energy,375000000,1,3.750000e+08
383,Ace Investments,339000000,1,3.390000e+08
382,A123 Systems,339000000,1,3.390000e+08
403,Apple Ventures,319000000,1,3.190000e+08


*Which funding round was the most popular? Which was the least popular?* 

* `Series A` is the most popular funding round while `crowdfunding` is the least. [Series A are funding rounds for earlier stage companies and range on average between USD 1M–30M; crowdfunding allows investment in exchange for equity or product](https://support.crunchbase.com/hc/en-us/articles/115010458467-Glossary-of-Funding-Types).

In [23]:
q6 = '''
    SELECT 
        funding_round_type,
        COUNT(funding_round_type)
    FROM crunch
    WHERE raised_amount_usd != 0
    GROUP BY 1
    ORDER BY 2 DESC;
'''

df6 = pd.read_sql(q6, conn)
df6

,funding_round_type,COUNT(funding_round_type)
0,series-a,13377
1,series-c+,10764
2,series-b,8630
3,venture,8031
4,angel,7190
5,other,934
6,private-equity,312
7,post-ipo,29
8,crowdfunding,4


## Conclusion 

In this guided project, I analyzed startup investments from [Crunchbase](https://www.crunchbase.com/) while working with the following:
* a memory constraint of only 10 MB
* the SQL/pandas workflow for processing large data sets

I first reduced the overall memory footprint of the data set by 82.6% through:
* removal of data columns not useful for analysis and with missing values greater than 90%
* casting of data columns of low cardinality into the categorical type
* parsing of appropriate data columns into `datetime` type
* cleaning and casting of string columns into the categorical type
* casting of float columns to the integer type

I then loaded the modified data set by chunks into an SQLite database table and used the SQL/pandas workflow to analyze startup investments from Crunchbase. Using the said workflow, I provided the following analyses:
* Proportion of the total amount of funds that the top/bottom 10% and 1% companies raised
* Category of startup companies that attracted the most investment
* Investor which contributed the most money across all startups
* Investors which contributed the most money per startup
* Most/least popular funding round